### Histogram of observations and SMRT simulations with and without atmosphere for C087 and C090, high, low and base simulations:

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.patches import Patch

In [ ]:
%load_ext autoreload
%aimport aesop
%autoreload 1

Load observations:

In [ ]:
obs_c087 = aesop.get_obs_per_aoi('C087')
obs_c090 = aesop.get_obs_per_aoi('C090')

Load SMRT results without atmosphere:

In [ ]:
# UNCOMMENT TO GENERATE RESULTS
#from run_smrt_with_realistic_atmosphere import run_smrt_with_atmosphere, tb_results
#res_no_atmos = run_smrt_with_atmosphere('C087', extremes=True, skip_atmos=True)
#tb_results(res_no_atmos, save=True, filename='../Other/smrt_results_atmos/smrt_tb_no_atmosphere.nc')

Load SMRT results without atmosphere:


In [ ]:
smrt_no_atmos = xr.open_dataset('../Other/smrt_results_atmos/smrt_tb_no_atmosphere.nc')

Load SMRT results with atmosphere:

In [ ]:
smrt_c087 = xr.open_dataset('../Other/smrt_results_atmos/smrt_atmos_tb_C087.nc')
smrt_c090 = xr.open_dataset('../Other/smrt_results_atmos/smrt_atmos_tb_C090.nc')

Load pits with topography:

In [ ]:
pits = aesop.get_pit_topography()

In [ ]:
def set_title(ax, channel):
    if channel in ['M16-89', 'M17-157']:
        ax.set_title(channel[4:]+' GHz')
    elif channel == '118+5.0':
        ax.set_title('118$\pm$5.0 GHz')
    elif channel == 'M20-183+7':
        ax.set_title('183$\pm$7 GHz')
    elif channel == '243':
        ax.set_title('243 GHz')

In [ ]:
plt.rcParams.update({'font.size': 14})

fig, axes = plt.subplots(1,5, figsize=(16,6))

for ax, channel in zip(axes.flatten(), ['M16-89', '118+5.0', 'M17-157', 'M20-183+7', '243']):
    
    tb_all = obs_c087[channel].dropna().TB.values # Unhash these lines to only use data for C087.
    smrt_all = np.concatenate([smrt_c087.sel(channel=channel)['base'].values,
                               smrt_c087.sel(channel=channel)['high'].values,
                               smrt_c087.sel(channel=channel)['low'].values])
    smrt_all_no_atmos = np.concatenate([smrt_no_atmos.sel(channel=channel)['base'].values,
                                        smrt_no_atmos.sel(channel=channel)['high'].values,
                                        smrt_no_atmos.sel(channel=channel)['low'].values])
    
    ax.hist(tb_all, histtype='step', stacked=True, fill=True, alpha=0.4, density=True, linewidth=1.2, color='#FF9933', label='Observations')
    ax.hist(smrt_all_no_atmos[~np.isnan(smrt_all_no_atmos)], histtype='step', stacked=True, fill=True, alpha=0.3, density=True, linewidth=1.2, color='#0080FF', label='SMRT no atmosphere')
    ax.hist(smrt_all[~np.isnan(smrt_all)], histtype='step', stacked=True, fill=True, alpha=0.3, density=True, linewidth=1.2, color='m', label='SMRT atmosphere')
    
    # Plot histogram median lines
    ax.axvline(np.quantile(tb_all,0.5), color='k', linestyle='dashed', linewidth=1, alpha=0.4)
    ax.axvline(np.quantile(smrt_all_no_atmos[~np.isnan(smrt_all_no_atmos)],0.5), color='b', linestyle='dashed', linewidth=1, alpha=0.4)
    ax.axvline(np.quantile(smrt_all[~np.isnan(smrt_all)],0.5), color='r', linestyle='dashed', linewidth=1, alpha=0.4)
   
    print(np.quantile(tb_all,0.5),np.quantile(smrt_all_no_atmos[~np.isnan(smrt_all_no_atmos)],0.5),np.quantile(smrt_all[~np.isnan(smrt_all)],0.5))
    
    ax.set_xlabel('TB (K)')
    
    set_title(ax, channel)

legend_labels = {'#FF9933': 'Observations', '#0080FF': 'SMRT no atmosphere', 'magenta': 'SMRT atmosphere'}

patches = [Patch(color=color, label=label, alpha=0.3)
           for color, label in legend_labels.items()]

axes[3].legend(handles=patches,
          bbox_to_anchor=(1.1, 1.16),
          facecolor='white', ncol=3)

axes[0].set_ylabel('Normalized frequency * 100')
tick_spacing = 50

for ax in axes:
    ax.set_xlim([100,260])
    # after plotting the data, format the labels
    current_values = ax.get_yticks()
    # Multiply by 100
    new_values = np.round(current_values * 1e2,2)
    # using format string '{:.0f}' here but you can choose others
    ax.set_yticklabels([y for y in new_values])
    #ax.ticklabel_format(axis='y', scilimits=[0, 0])
    ax.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
fig.subplots_adjust(hspace=1.0)

#fig.savefig('../Figs/Fig10.png', bbox_inches = 'tight', format='png')#, pad_inches = 0)


plt.show()


In [ ]:
np.quantile(tb_all,0.5)

In [ ]:
np.quantile(smrt_all_no_atmos[~np.isnan(smrt_all_no_atmos)],0.5)

In [ ]:
np.quantile(smrt_all[~np.isnan(smrt_all)],0.5)